
## Análisis del Parque Vehicular



In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



## Carga de datos


In [2]:
df = pd.read_csv('../parque_vehicular_datos_abiertos_13NOV2018.csv', delimiter=';')


C:\Users\julia\AppData\Local\Temp\ipykernel_20552\2260371232.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../parque_vehicular_datos_abiertos_13NOV2018.csv', delimiter=';')



## Preprocesamiento


In [7]:
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include=['object']).drop(['TIPO_PLACA'], axis=1).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])
for col in categorical_features:
    df[col] = df[col].astype(str)
df = df.dropna()



##  K-Means


In [22]:
kmeans = Pipeline(steps=[('preprocessor', preprocessor),
                         ('cluster', KMeans(n_clusters=4, random_state=0))])
kmeans.fit(df)


C:\Users\julia\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['ANIO_DE_FABRICACION', 'CANTIDAD_DE_CILINDROS', 'CANTIDAD_DE_PUERTAS',
       'VALOR_DEL_VEHICULO', 'IMP_VALOR_DEL_VEHICULO', 'ANIO_INGRESO',
       'MES_INGRESO', 'CAPACIDAD'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['CILINDRADA', 'COLORES', 'FECHA_DE_IMPORTACION', 'FECHA_DE_INGRESO',
       'CLASE', 'PERTENENCIA', 'MARCA', 'MODELO', 'DES_CAPACIDAD',
       'COMBUSTIBLE', 'ADUANA', 'CONDICION_INGRESO',
       'PROPIETARIO_DEPARTAMENTO', 'PROPIETARIO_MUNICIPIO', 'ESTADO'],
      dtype='object'))])),
                ('cluster', KMeans(n_clusters=4, random_state=0))])

In [23]:
df['Cluster'] = kmeans.predict(df)


In [24]:
numeric_cols = df.select_dtypes(include=['float64', 'int32']).columns

df_grouped_mean = df[numeric_cols].groupby('Cluster').mean()

print(df_grouped_mean)


         ANIO_DE_FABRICACION  CANTIDAD_DE_CILINDROS  CANTIDAD_DE_PUERTAS  \
Cluster                                                                    
0                2007.785663               4.288286             3.835526   
1                1983.788755               0.499768             0.917221   
2                2002.000000               4.000000         41406.000000   
3                2007.579956               5.485616             2.901036   

         VALOR_DEL_VEHICULO  IMP_VALOR_DEL_VEHICULO  ANIO_INGRESO  \
Cluster                                                             
0               6212.739204             6214.762955   2012.247559   
1               3431.914821             3467.938105   1991.685358   
2               2008.100000             2008.100000   2009.000000   
3              20680.292590            22916.619786   2010.559105   

         MES_INGRESO  CAPACIDAD  
Cluster                          
0           6.669305   4.784245  
1           6.349980   4.5


1.  **Año de Fabricación (`ANIO_DE_FABRICACION`)**:
    
    -   El Cluster 1 agrupa los vehículos más antiguos en promedio, con un año de fabricación alrededor de 1983.
    -   Los Clusters 0, 2 y 3 parecen tener vehículos más modernos, con años de fabricación promedio cercanos o posteriores al año 2000.
2.  **Cantidad de Cilindros (`CANTIDAD_DE_CILINDROS`)** y **Cantidad de Puertas (`CANTIDAD_DE_PUERTAS`)**:
    
    -   Los vehículos en el Cluster 1 tienen en promedio menos cilindros y puertas, lo que podría indicar que son modelos más simples o compactos.
    -   El Cluster 2 destaca con un número extremadamente alto en la cantidad de puertas (41,406), lo cual es atípico y sugiere un posible error en los datos o una interpretación errónea de la variable.
    -   Los Clusters 0 y 3 presentan una cantidad promedio de cilindros y puertas que son más típicas de vehículos de uso personal.
3.  **Valor del Vehículo (`VALOR_DEL_VEHICULO`)** y **Valor en Importación (`IMP_VALOR_DEL_VEHICULO`)**:
    
    -   El Cluster 3 tiene el valor promedio más alto tanto en el valor del vehículo como en el valor de importación, lo que sugiere que estos vehículos podrían ser de un segmento premium o lujo.
    -   El Cluster 1 presenta el valor promedio más bajo, lo cual es consistente con vehículos más antiguos y posiblemente de menor valor.
4.  **Mes de Ingreso (`MES_INGRESO`)** y **Capacidad (`CAPACIDAD`)**:
    
    -   Los valores promedio del mes de ingreso no parecen ofrecer diferenciaciones significativas entre los clusters.
    -   La capacidad promedio es notablemente más alta en el Cluster 3, lo que puede indicar vehículos más grandes o con mayor capacidad de carga.

### Análisis de Frecuencia para Variables Categóricas

In [26]:
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(df.groupby(['Cluster', col])[col].size().unstack(fill_value=0))


TIPO_PLACA  ALQUILER  AUTOBUS  CAMION  CRUZ ROJA  CUERPO CONSULAR  \
Cluster                                                             
0               3432        0     248          7               19   
1               4169     5100   22627          2                3   
2                  0        0       0          0                0   
3                  7    12389   28442         39               27   

TIPO_PLACA  CUERPO DIPLOMATICO  DISCAPACITADOS  MICROBUS(TRANSPORTE)  \
Cluster                                                                
0                          573              57                    34   
1                            6              21                   175   
2                            0               0                     0   
3                          167              36                  4422   

TIPO_PLACA  MISION INTERNACIONAL  MOTOCICLETA  NACIONAL  OFICIAL  PARTICULAR  \
Cluster                                                                 


 **Clusters de Vehículos por Tipo de Uso**:
    
    -   El Cluster 1 destaca por una alta frecuencia de vehículos con placas de 'ALQUILER' y 'AUTOBUS', lo que sugiere que este grupo podría estar compuesto principalmente por vehículos comerciales o de transporte público.
    -   El Cluster 3 tiene una gran cantidad de vehículos con placas de 'CAMION', indicando que este cluster probablemente incluye vehículos de carga o de uso industrial.
    -   El Cluster 0 tiene una cantidad significativa de vehículos con placas 'PARTICULAR', lo que sugiere que estos son principalmente vehículos de uso personal.
 **Vehículos de Organizaciones Especiales**:
    
    -   El Cluster 3 también muestra una cantidad notable de vehículos con placas de 'CUERPO DIPLOMATICO' y 'CRUZ ROJA', indicando que estos pueden ser vehículos utilizados por entidades internacionales o de ayuda humanitaria.


 **Predominancia de Vehículos Particulares**:
    
    -   Los Clusters 0 y 1 contienen la mayoría de los vehículos con placas 'PARTICULAR', reafirmando que son predominantemente vehículos personales.
**Especialización del Cluster 1**:
    
    -   El Cluster 1 también muestra un número considerable de vehículos con placas 'MOTOCICLETA', lo que podría sugerir que este cluster se especializa en vehículos de dos ruedas, además de su papel en el transporte.
**Variabilidad del Cluster 0**:
    
    -   El Cluster 0 parece ser el más variado, con vehículos de 'MISION INTERNACIONAL', 'MOTOCICLETA', y 'NACIONAL' presentes en menor medida, sugiriendo una mezcla de diferentes tipos de vehículos.
  **Exclusividad de Ciertos Clusters**:
    
    -   Algunas categorías como 'DISCAPACITADOS' y 'REMOQUE' están casi exclusivamente representadas en el Cluster 1, lo que podría



### Conclusiones Generales:

-   El **Cluster 1** parece representar vehículos antiguos y posiblemente más económicos.
-   El **Cluster 3** se caracteriza por vehículos con una mayor capacidad y valor, lo que podría asociarse a vehículos más nuevos y posiblemente de gama alta.
-   El **Cluster 2** presenta una anomalía en la cantidad de puertas que necesita ser investigada, ya que el número reportado no parece ser realista para vehículos comunes.
-   El **Cluster 0** parece ser el más equilibrado en términos de las características medias, sugiriendo que podría contener una variedad de vehículos de uso estándar.

- Los clusters parecen estar segmentados en función del tipo de uso del vehículo, con claras diferencias entre uso personal, comercial, especializado y de servicio.
- El Cluster 1 es notable por su alta concentración de vehículos de alquiler y autobuses, así como motocicletas, lo que sugiere una posible focalización en vehículos destinados a servicios o de menor tamaño.
- El Cluster 3 parece estar relacionado con vehículos de carga pesada y servicios especiales o diplomáticos, lo que indica un uso más industrial o gubernamental.